In [2]:
import pandas as pd
from functools import reduce

In [3]:
df_attributes = pd.read_csv('../attribute_extraction_metadata_template/attribute_definition_top25L2_filtered_augmented_redacted_addedsinglemulti_01182023.csv')
df_attributes_orig = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')
cats = set(df_attributes['category'])
df_attributes_orig_sub = df_attributes_orig[
    (df_attributes_orig.category.apply(lambda x: x in cats)) & (df_attributes_orig['entry mode'].apply(lambda x: x != 'free_text'))
]
df_attributes_orig_sub_group = df_attributes_orig_sub[['attribute_field', 'category_attributevalue']].groupby('attribute_field').agg({
    'category_attributevalue': lambda x: sorted(set(reduce(lambda x, y: x+y, [[str(j) for j in eval(i)] for i in x])))
}).reset_index()


attr_vals_orig_normalized = {}
for i in df_attributes_orig_sub_group.to_dict('records'):
    attr_vals_orig_normalized[i['attribute_field']] = set(i['category_attributevalue'])

all_orig_kvpairs = set()

for k in attr_vals_orig_normalized:
    for v in attr_vals_orig_normalized[k]:
        all_orig_kvpairs.add((k, v))

In [4]:
len(all_orig_kvpairs)

6875

In [6]:
all_orig_kvpairs = sorted(all_orig_kvpairs)

In [9]:
any([(i[0][0] == " " or i[0][-1] == " " or i[1][0] == " " or i[1][-1] == " ") for i in all_orig_kvpairs])

False

In [13]:
any([('|' in i[0] or '|' in i[1]) for i in all_orig_kvpairs])

False

In [11]:
all_orig_kvpairs_text = [i[0] + '|' + i[1] for i in all_orig_kvpairs]

In [15]:
with open('25L2_unfreetext_attribute_name_value_pairs_02232023.txt', 'w') as f:
    for i in all_orig_kvpairs_text:
        f.write(i + '\n')